# Super Store Analysis

---

## Installing dependencies

In [261]:
# !pip install xlrd
# !pip install pandas
# !pip install numpy
# !pip install mysql-connector-python
# !pip install sqlalchemy
# !pip install pymysql

---

## Setting up the environment

In [262]:
import re
import os
import warnings
import pandas as pd
import numpy as np
import mysql.connector as mysql
from sqlalchemy import create_engine, text
from sqlalchemy.exc import *
from google.cloud import bigquery as bq
from google.oauth2 import service_account

warnings.filterwarnings('ignore')
# show max columns
pd.set_option('display.max_columns', None)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'explore29-33756158108f.json'

---

## Extract

In [263]:
file_path = 'data-src/Sample - Superstore.xls'
orders_df = pd.read_excel(file_path, sheet_name='Orders', header=0)
people_df = pd.read_excel(file_path, sheet_name='People', header=0)
returns_df = pd.read_excel(file_path, sheet_name='Returns', header=0)

---

## Transform

### Orders

#### Overview

In [264]:
orders_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,State/Province,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,US-2021-103800,2021-01-03,2021-01-07,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,Texas,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2021-141817,2021-01-05,2021-01-12,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,Pennsylvania,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840


In [265]:
orders_df.tail()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,State/Province,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
10189,10190,US-2024-143259,2024-12-30,2025-01-03,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,New York,10009,East,OFF-BI-10003684,Office Supplies,Binders,Wilson Jones Legal Size Ring Binders,52.776,3,0.2,19.7910
10190,10191,US-2024-115427,2024-12-30,2025-01-03,Standard Class,EB-13975,Erica Bern,Corporate,United States,Fairfield,California,94533,West,OFF-BI-10004632,Office Supplies,Binders,GBC Binding covers,20.720,2,0.2,6.4750
10191,10192,US-2024-156720,2024-12-30,2025-01-03,Standard Class,JM-15580,Jill Matthias,Consumer,United States,Loveland,Colorado,80538,West,OFF-FA-10003472,Office Supplies,Fasteners,Bagged Rubber Bands,3.024,3,0.2,-0.6048
10192,10193,US-2024-143259,2024-12-30,2025-01-03,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,New York City,New York,10009,East,TEC-PH-10004774,Technology,Phones,Gear Head AU3700S Headset,90.930,7,0.0,2.7279
10193,10194,CA-2024-143500,2024-12-30,2025-01-03,Standard Class,HO-15230,Harry Olson,Consumer,Canada,Charlottetown,Prince Edward Island,C0A,East,OFF-BI-10004040,Office Supplies,Binders,Wilson Jones Impact Binders,3.024,3,0.2,-0.6048


In [266]:
orders_df.shape

(10194, 21)

In [267]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10194 entries, 0 to 10193
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Row ID          10194 non-null  int64         
 1   Order ID        10194 non-null  object        
 2   Order Date      10194 non-null  datetime64[ns]
 3   Ship Date       10194 non-null  datetime64[ns]
 4   Ship Mode       10194 non-null  object        
 5   Customer ID     10194 non-null  object        
 6   Customer Name   10194 non-null  object        
 7   Segment         10194 non-null  object        
 8   Country/Region  10194 non-null  object        
 9   City            10194 non-null  object        
 10  State/Province  10194 non-null  object        
 11  Postal Code     10194 non-null  object        
 12  Region          10194 non-null  object        
 13  Product ID      10194 non-null  object        
 14  Category        10194 non-null  object        
 15  Su

---

#### Renaming headers

In [268]:
def snake_case(data:str) -> str:
	return(data.lower().strip().replace(' ','_').replace('-', '_'))

In [269]:
orders_df = orders_df.rename(columns={
	'Country/Region': 'country',
    'State/Province': 'province',
    'Postal Code': 'post_code'
})

orders_df.columns = [snake_case(col) for col in orders_df.columns]

---

#### Cleaning

**Check for null values**

In [270]:
print(orders_df.isnull().sum())

row_id           0
order_id         0
order_date       0
ship_date        0
ship_mode        0
customer_id      0
customer_name    0
segment          0
country          0
city             0
province         0
post_code        0
region           0
product_id       0
category         0
sub_category     0
product_name     0
sales            0
quantity         0
discount         0
profit           0
dtype: int64


---

**Check for duplicates**

In [271]:
print(orders_df.duplicated(subset=['order_id', 'product_id'], keep='last').sum())

11


**Show duplicates**

In [272]:
orders_df[orders_df.duplicated(subset=['order_id', 'product_id'], keep=False)].head(4)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
390,391,US-2021-150119,2021-04-23,2021-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588
391,392,US-2021-150119,2021-04-23,2021-04-27,Standard Class,LB-16795,Laurel Beltran,Home Office,United States,Columbus,Ohio,43229,East,FUR-CH-10002965,Furniture,Chairs,Global Leather Highback Executive Chair with P...,281.372,2,0.3,-12.0588
1698,1699,CA-2021-153623,2021-11-24,2021-12-05,Standard Class,JP-11135,James Peterman,Corporate,Canada,St. John's,Newfoundland and Labrador,A0A,East,FUR-FU-10002501,Furniture,Furnishings,Nu-Dell Executive Frame,99.120,8,0.0,35.4144
1699,1700,CA-2021-153623,2021-11-24,2021-12-05,Standard Class,JP-11135,James Peterman,Corporate,Canada,St. John's,Newfoundland and Labrador,A0A,East,FUR-FU-10002501,Furniture,Furnishings,Nu-Dell Executive Frame,99.120,8,0.0,35.4144


In [273]:
orders_df[orders_df.duplicated(subset=['order_id', 'product_id'], keep=False)].tail(4)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
9384,9385,US-2024-152912,2024-11-09,2024-11-12,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,1633.140,9,0.0,473.6106
9385,9386,US-2024-152912,2024-11-09,2024-11-12,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,544.380,3,0.0,157.8702
9800,9801,US-2024-118017,2024-12-03,2024-12-06,Second Class,LC-16870,Lena Cacioppo,Consumer,United States,Thornton,Colorado,80229,West,TEC-AC-10002006,Technology,Accessories,Memorex Micro Travel Drive 16 GB,76.752,6,0.2,10.5534
9801,9802,US-2024-118017,2024-12-03,2024-12-06,Second Class,LC-16870,Lena Cacioppo,Consumer,United States,Thornton,Colorado,80229,West,TEC-AC-10002006,Technology,Accessories,Memorex Micro Travel Drive 16 GB,102.336,8,0.2,14.0712


**Problem:**

Upon closer inspection, it is found that the duplicate columns have the same/different ```quantity```, which affects the ```sales``` and ```profit``` columns. Hence, fiest rows with all identical columns will be removed, while rows that share identical data across each column but have different ```quantity``` data will be merged.

In [274]:
orders_df = orders_df.drop_duplicates(subset=['order_id', 'product_id', 'quantity'], keep='last')

merged_df = orders_df.groupby(['order_id', 'product_id'], as_index=False).agg({
	'sales': 'sum',
	'quantity': 'sum',
	'profit': 'sum'
})

orders_df[(9384 <= orders_df['row_id']) & ( orders_df['row_id'] <= 9386)]

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
9383,9384,US-2024-159597,2024-11-09,2024-11-14,Standard Class,MC-17590,Matt Collister,Corporate,United States,Coachella,California,92236,West,OFF-ST-10002289,Office Supplies,Storage,"Safco Wire Cube Shelving System, For Use as 4 ...",63.56,2,0.0,3.1780
9384,9385,US-2024-152912,2024-11-09,2024-11-12,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,1633.14,9,0.0,473.6106
9385,9386,US-2024-152912,2024-11-09,2024-11-12,Second Class,BM-11650,Brian Moss,Corporate,United States,Columbia,Maryland,21044,East,OFF-ST-10003208,Office Supplies,Storage,Adjustable Depth Letter/Legal Cart,544.38,3,0.0,157.8702


In [275]:
orders_df.shape

(10192, 21)

---

#### Final inspection

In [276]:
display(
	orders_df.info(),
	orders_df.head()
)

<class 'pandas.core.frame.DataFrame'>
Index: 10192 entries, 0 to 10193
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   row_id         10192 non-null  int64         
 1   order_id       10192 non-null  object        
 2   order_date     10192 non-null  datetime64[ns]
 3   ship_date      10192 non-null  datetime64[ns]
 4   ship_mode      10192 non-null  object        
 5   customer_id    10192 non-null  object        
 6   customer_name  10192 non-null  object        
 7   segment        10192 non-null  object        
 8   country        10192 non-null  object        
 9   city           10192 non-null  object        
 10  province       10192 non-null  object        
 11  post_code      10192 non-null  object        
 12  region         10192 non-null  object        
 13  product_id     10192 non-null  object        
 14  category       10192 non-null  object        
 15  sub_category   10192 non

None

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit
0,1,US-2021-103800,2021-01-03,2021-01-07,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,Texas,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512
1,2,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870
2,3,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717
3,4,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748
4,5,US-2021-141817,2021-01-05,2021-01-12,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,Pennsylvania,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840


---

### People

#### Overview

In [277]:
people_df.head()

,Regional Manager,Region
0,Sadie Pawthorne,West
1,Chuck Magee,East
2,Roxanne Rodriguez,Central
3,Fred Suzuki,South


In [278]:
people_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Regional Manager  4 non-null      object
 1   Region            4 non-null      object
dtypes: object(2)
memory usage: 196.0+ bytes


---

#### Renaming headers

In [279]:
people_df = people_df.rename(columns={
	'Regional Manager': 'manager',
    'Region': 'region'
})

---

#### Final inspection

In [280]:
people_df.tail()

,manager,region
0,Sadie Pawthorne,West
1,Chuck Magee,East
2,Roxanne Rodriguez,Central
3,Fred Suzuki,South


---

### Returns

#### Overview

In [281]:
returns_df.head()

,Returned,Order ID
0,Yes,US-2021-100762
1,Yes,US-2021-100762
2,Yes,US-2021-100762
3,Yes,US-2021-100762
4,Yes,US-2021-100867


In [282]:
returns_df.tail()

,Returned,Order ID
795,Yes,US-2024-147886
796,Yes,US-2024-147998
797,Yes,US-2024-151127
798,Yes,US-2024-155999
799,Yes,US-2024-155999


In [283]:
returns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Returned  800 non-null    object
 1   Order ID  800 non-null    object
dtypes: object(2)
memory usage: 12.6+ KB


---

#### Renaming headers

In [284]:
returns_df.columns = [snake_case(col) for col in returns_df.columns]

---

#### Cleaning


**Checking for null values**

In [285]:
returns_df.isnull().sum()

returned    0
order_id    0
dtype: int64

**Checking for duplicates**

In [286]:
print(returns_df.duplicated().sum())

504


**Show duplicates**

In [287]:
returns_df[returns_df.duplicated()].head(10)

,returned,order_id
1,Yes,US-2021-100762
2,Yes,US-2021-100762
3,Yes,US-2021-100762
6,Yes,US-2021-102652
7,Yes,US-2021-102652
8,Yes,US-2021-102652
11,Yes,US-2021-103744
13,Yes,US-2021-103940
14,Yes,US-2021-103940
15,Yes,US-2021-103940


In [288]:
compare_orders_df = orders_df.loc[orders_df['order_id'] == 'US-2021-100762', ['order_id', 'order_date', 'ship_date', 'product_id', 'customer_id']]
compare_orders_df

,order_id,order_date,ship_date,product_id,customer_id
1684,US-2021-100762,2021-11-24,2021-11-29,OFF-AR-10000380,NG-18355
1688,US-2021-100762,2021-11-24,2021-11-29,OFF-LA-10003930,NG-18355
1690,US-2021-100762,2021-11-24,2021-11-29,OFF-PA-10001815,NG-18355
1691,US-2021-100762,2021-11-24,2021-11-29,OFF-PA-10004082,NG-18355


In [289]:
# get the number of times each order_id appeared in orders
orders_oid_count = orders_df['order_id'].value_counts().reset_index()
orders_oid_count.columns = ['order_id', 'orders_oid_count']

# get the number of times each order_id appeared in returns
returns_oid_count = returns_df['order_id'].value_counts().reset_index()
returns_oid_count.columns = ['order_id', 'returns_oid_count']

# merge the two df and compare the number of times each order_id appeared in both df
# counting only the order_id that appeared in both df
same_id_entries = orders_oid_count.merge(returns_oid_count, on='order_id', how='inner')

display(same_id_entries)

mismatch = len(same_id_entries[same_id_entries['orders_oid_count'] != same_id_entries['returns_oid_count']])
print(f'Mismatch: {mismatch}')

,order_id,orders_oid_count,returns_oid_count
0,US-2024-100111,14,14
1,US-2023-165330,11,11
2,US-2022-164882,9,9
3,US-2024-161956,8,8
4,US-2021-142769,8,8
...,...,...,...
291,US-2024-137099,1,1
292,US-2024-142328,1,1
293,US-2024-108294,1,1
294,US-2024-117926,1,1


Mismatch: 0


**Conclusion:**

- Each ```order_id``` that appeared in ```Returns``` appeared the same amount of time in ```Orders```. Hence, we can conclude that ```Returns``` is the 'return status' for each product in each order. 

- Hence, we can merge ```Returns``` into ```Orders```.

- However, upon further inspection the ```Returns``` table only contains 'Yes', hence we need to set 'No' to the ```product_id``` that has not been returned.

In [290]:
len(returns_df[returns_df['returned'] != 'Yes'])

0

---

#### Final inspection

In [291]:
returns_df.tail()

,returned,order_id
795,Yes,US-2024-147886
796,Yes,US-2024-147998
797,Yes,US-2024-151127
798,Yes,US-2024-155999
799,Yes,US-2024-155999


---

### Joining and splitting

#### Entity Relationship Diagram

<img src="https://github.com/user-attachments/assets/99a8505b-cc9a-4d7d-96db-6410d922f0ee" alt="Image" width="800" height="450">

This step splits data into smaller dataframe, which represent different tables in the schema.

#### ```orders``` table:

**Merging ```returns``` with ```orders```**

In [292]:
# this works for the first time because the number of order_id that appears in both df appears for the same amount of time
# when doing ochestration more safe checks will be implemented
orders_df['returned'] = orders_df['order_id'].isin(returns_df['order_id']).map({True: 'Yes', False: 'No'})
orders_df.head()
orders_df[orders_df['order_id'] == 'US-2021-100762']

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,returned
1684,1685,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-AR-10000380,Office Supplies,Art,"Hunt PowerHouse Electric Pencil Sharpener, Blue",151.92,4,0.0,45.5760,Yes
1688,1689,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-LA-10003930,Office Supplies,Labels,"Dot Matrix Printer Tape Reel Labels, White, 50...",196.62,2,0.0,96.3438,Yes
1690,1691,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-PA-10001815,Office Supplies,Paper,Xerox 1885,144.12,3,0.0,69.1776,Yes
1691,1692,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-PA-10004082,Office Supplies,Paper,Adams Telephone Message Book w/Frequently-Call...,15.96,2,0.0,7.9800,Yes


**Checking the merge**

In [293]:
orders_df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,returned
0,1,US-2021-103800,2021-01-03,2021-01-07,Standard Class,DP-13000,Darren Powers,Consumer,United States,Houston,Texas,77095,Central,OFF-PA-10000174,Office Supplies,Paper,"Message Book, Wirebound, Four 5 1/2"" X 4"" Form...",16.448,2,0.2,5.5512,No
1,2,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-BI-10004094,Office Supplies,Binders,GBC Standard Plastic Binding Systems Combs,3.540,2,0.8,-5.4870,No
2,3,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-LA-10003223,Office Supplies,Labels,Avery 508,11.784,3,0.2,4.2717,No
3,4,US-2021-112326,2021-01-04,2021-01-08,Standard Class,PO-19195,Phillina Ober,Home Office,United States,Naperville,Illinois,60540,Central,OFF-ST-10002743,Office Supplies,Storage,SAFCO Boltless Steel Shelving,272.736,3,0.2,-64.7748,No
4,5,US-2021-141817,2021-01-05,2021-01-12,Standard Class,MB-18085,Mick Brown,Consumer,United States,Philadelphia,Pennsylvania,19143,East,OFF-AR-10003478,Office Supplies,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,19.536,3,0.2,4.8840,No


In [294]:
orders_df[orders_df['order_id'] == 'US-2021-100762']

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,province,post_code,region,product_id,category,sub_category,product_name,sales,quantity,discount,profit,returned
1684,1685,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-AR-10000380,Office Supplies,Art,"Hunt PowerHouse Electric Pencil Sharpener, Blue",151.92,4,0.0,45.5760,Yes
1688,1689,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-LA-10003930,Office Supplies,Labels,"Dot Matrix Printer Tape Reel Labels, White, 50...",196.62,2,0.0,96.3438,Yes
1690,1691,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-PA-10001815,Office Supplies,Paper,Xerox 1885,144.12,3,0.0,69.1776,Yes
1691,1692,US-2021-100762,2021-11-24,2021-11-29,Standard Class,NG-18355,Nat Gilpin,Corporate,United States,Jackson,Michigan,49201,Central,OFF-PA-10004082,Office Supplies,Paper,Adams Telephone Message Book w/Frequently-Call...,15.96,2,0.0,7.9800,Yes


In [299]:
orders = orders_df[['order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'product_id', 'sales', 'quantity', 'discount', 'profit', 'returned']]

# compare the number of entries in the raw data to the current orders df
print(f'Dimensions: {orders.shape}')
print(f'Must have: ({orders_df.shape[0]}, x)\n')

orders.info()

Dimensions: (10192, 11)
Must have: (10192, x)

<class 'pandas.core.frame.DataFrame'>
Index: 10192 entries, 0 to 10193
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   order_id     10192 non-null  object        
 1   order_date   10192 non-null  datetime64[ns]
 2   ship_date    10192 non-null  datetime64[ns]
 3   ship_mode    10192 non-null  object        
 4   customer_id  10192 non-null  object        
 5   product_id   10192 non-null  object        
 6   sales        10192 non-null  float64       
 7   quantity     10192 non-null  int64         
 8   discount     10192 non-null  float64       
 9   profit       10192 non-null  float64       
 10  returned     10192 non-null  object        
dtypes: datetime64[ns](2), float64(3), int64(1), object(5)
memory usage: 955.5+ KB


In [296]:
print(list(orders_df.columns))

['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'customer_name', 'segment', 'country', 'city', 'province', 'post_code', 'region', 'product_id', 'category', 'sub_category', 'product_name', 'sales', 'quantity', 'discount', 'profit', 'returned']


**```customers``` table**

Note: this table should only contain unique customer data (unique ```customer_id```).

In [297]:
customers = orders_df[['customer_id', 'customer_name', 'segment', 'country', 'city', 'province', 'post_code', 'region']].drop_duplicates(subset=['customer_id'])

# compare the number of unique customer_id in the raw data to the current df
print(f'Dimensions: {customers.shape}')
print(f'Must have: ({len(orders_df['customer_id'].unique())}, x)\n')

customers.info()

Dimensions: (804, 8)
Must have: (804, x)

<class 'pandas.core.frame.DataFrame'>
Index: 804 entries, 0 to 9321
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    804 non-null    object
 1   customer_name  804 non-null    object
 2   segment        804 non-null    object
 3   country        804 non-null    object
 4   city           804 non-null    object
 5   province       804 non-null    object
 6   post_code      804 non-null    object
 7   region         804 non-null    object
dtypes: object(8)
memory usage: 56.5+ KB
